# Machens-Brody 2D discrimination model

We implement and simulate from the 2005 Machens-Romo-Brody 2 population mean field model 
Using the matlab code to simulate, with initpar as is, except with 
- large additive noise (par.mfnoise=1.5)
- using only the dynamics during the "Comparison/Decision making" phase (TSO2 = 0, TSF2 = 2000, T = 2000)
- Subsampling the near-continous dynamics to simulate discretised measurement process ($\Delta t_{ODE}$ = 0.1 msec, $\Delta t_{meas}$ = 1 msec, or even more subsampled)
- Only ambigous trials, type [4,4]



In [1]:
%run 'GPDM_direct_fixedpoints.ipynb'

/nfs/nhome/live/gbohner/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



In [2]:
import scipy.optimize
import pickle
import pickle, datetime, time

In [3]:
import scipy.io

In [4]:
tmp = scipy.io.loadmat('../Matlab/Machens_Brody/twonode/MachensBrodySim_20171205T004721.mat', squeeze_me=True)

In [5]:
simParams = OrderedDict()
simParams['seed'] = 1234
simParams['Ntrain'] = 20
simParams['delta_t'] = 5
simParams['Sigma_y'] = 1e-2
simParams['rescale'] = 1e3
np.random.seed(simParams['seed'])
x = tmp['y_python'][:,0::simParams['delta_t'],:simParams['Ntrain']]*simParams['rescale'] # Rescaling to reduce numerical instability
y = x + np.sqrt(simParams['Sigma_y'])*np.random.randn(*x.shape)
y.shape

(2, 40, 20)

In [6]:
# Plot the data
plots_by_run = []
for v in range(y.shape[2]):
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(y[0,:,v]), 
                      y=np.squeeze(y[1,:,v]), 
                      mode='lines')
    )
    
plt(plots_by_run)

# Initialise the model

In [70]:
# Try to solve the full problem (or some parts of it)
D = 2
Nz = 36
Ns = 6

#######################################################
# Initialise the parameters
paramdict = init_params(y, D, Nz, Ns)

# Re-initialise / add some more parameters
paramdict['Sigma_s'] = 1e-2*np.ones((Ns,1))
paramdict['Sigma_J'] = 1e-2*np.ones((Ns*D,1))

# Add transformations for certain parameters 
# (Note that the parameter indices may be changed by transforms! (for cholesky repres of matrices, "SquareMatrix" type))
transforms = OrderedDict()
for par in ['Sigma_0_0', 'Sigma_u', 'Sigma_s', 'Sigma_J', 'lengthscales', 'Sigma_eps', 'Sigma_nu', 'kernel_variance']:
    transforms[par] = {}
    transforms[par]['type'] = "Square"

# Create vectorised and transformed representation
(init_paramvec, dict_ind, dict_shape) = params_to_vec(paramdict, transforms=transforms)

#######################################################
# Optimise only certain elements of paramvec (messy with indices)
opt_params = np.arange(init_paramvec.shape[0])
# opt_params = np.delete(opt_params, np.hstack([dict_ind['C'], dict_ind['Sigma_nu'], dict_ind['J']])) # All except the ones listed here
opt_params = np.delete(opt_params, np.hstack([dict_ind['C']])) # All except the ones listed here
cur_pvec = init_paramvec[opt_params]

                                              
#######################################################
# Add bounds for parameters 
bnds = list(((None, None),) * init_paramvec.shape[0])
                                              
bnds_final = []
for i in opt_params:
    bnds_final.append(bnds[i])
bnds = tuple(bnds_final)

#######################################################
# Add priors (to span at least the bounds)
priors = []

# Add prior to ensure inducing points are smooth
cur_prior = {}
cur_prior['type'] = "InducingSmooth_and_DPP"
cur_prior['metadata'] = {}
def unpack_dict_tmp(pdict):
    kernelparams = {'lengthscales': pdict['lengthscales'], 'kernel_variance': pdict['kernel_variance']}
    # Return the parameters we want in the required format (joint smoothness of (z-u) and s)
    return (np.concatenate([pdict['u'], pdict['s']],axis=1), 
            np.concatenate([pdict['z'], pdict['s']],axis=1),
            np.concatenate([pdict['Sigma_u'], pdict['Sigma_s']]),
            kernelparams)
cur_prior['metadata']['unpack_dict'] = unpack_dict_tmp
cur_prior['metadata']['kernel_func'] = RBF
cur_prior['metadata']['prior_weight_Smooth'] = 1e0
cur_prior['metadata']['prior_weight_DPP'] = 1e0
priors.append(cur_prior)


In [71]:
# Define a plotting function for callback that shows the current transition function estimate
import plotly.figure_factory

def machens_callback_plot_external(pvec_partial, 
                                  opt_params, init_paramvec, transforms, dict_ind, dict_shape
                                 ):
    
    paramvec = replace_params(pvec_partial, opt_params, init_paramvec)
    paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)
       
    # Unpack the usual parameters
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
        paramdict.values()[:12]
    
    if np.any(np.isnan(lengthscales)):
        set_trace()
    
    # Deal with the extra possible parameters
    Sigma_s = None; Sigma_J=None;
    if 'Sigma_s' in paramdict.keys():
        Sigma_s = paramdict['Sigma_s']
    if 'Sigma_J' in paramdict.keys():
        Sigma_J = paramdict['Sigma_J']

    # Plot transition function
    xtmp, ytmp = np.meshgrid(np.linspace(np.min(z[0,:]),np.max(z[0,:]),30),
                             np.linspace(np.min(z[1,:]),np.max(z[1,:]),30))
    xstar = np.concatenate([xtmp.flatten()[:,None], ytmp.flatten()[:,None]], axis=1).T

    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u=Sigma_u, sig_s=Sigma_s, sig_J = Sigma_J)
    mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

    #print(time_full_iter(pvec, y, D, Nz, Ns)[0])
    
    quiver_fig = plotly.figure_factory.create_quiver(np.squeeze(xstar[0,:]), 
                                                     np.squeeze(xstar[1,:]), 
                                                     np.squeeze(mu_star[0,:]-xstar[0,:]), 
                                                     np.squeeze(mu_star[1,:]-xstar[1,:]),
                                                    scale=.25,
                                                    arrow_scale=.4,)

#     quiver_fig = plotly.figure_factory.create_quiver(np.squeeze(y_t1_rsh[0,:]), 
#                                                  np.squeeze(y_t1_rsh[1,:]), 
#                                                  np.squeeze(y_t_rsh[0,:]-y_t1_rsh[0,:]), 
#                                                  np.squeeze(y_t_rsh[1,:]-y_t1_rsh[1,:]),
#                                                 scale=.25,
#                                                 arrow_scale=.4,)
    
    # Add points to figure
    
    # Inducing point locations
    quiver_fig['data'].append(
        plt_type.Scatter(x=np.atleast_1d(np.squeeze(z[0,:])), y=np.atleast_1d(np.squeeze(z[1,:])), 
                         mode='markers', name="Inducing loc", marker=dict(size=10)))
    
    # Estimated fixed points
    quiver_fig['data'].append(
        plt_type.Scatter(x=np.atleast_1d(np.squeeze(s[0,:])), y=np.atleast_1d(np.squeeze(s[1,:])), 
                         mode='markers', name="Fixed loc", marker=dict(size=14)))
        
    
    plt(quiver_fig)
    
def machens_callback_plot(pvec_partial): 
    machens_callback_plot_external(
        pvec_partial, 
        opt_params, init_paramvec, transforms, dict_ind, dict_shape)

In [72]:
# Visualise initial estimate of the transition function
machens_callback_plot(cur_pvec)

In [73]:
# %run "GPDM_direct_fixedpoints.ipynb"

In [ ]:
# Compute initial objective function
time_full_iter(replace_params(cur_pvec, opt_params, init_paramvec), 
                                                y, dict_ind, dict_shape, log_transformed=log_transformed)[0]

## Set up the optimisation

In [75]:
#######################################################
# Prepare the optimisation
tmp_func = lambda pvec_partial: (time_full_iter(replace_params(pvec_partial, opt_params, init_paramvec), 
                                            y, dict_ind, dict_shape, 
                                            transforms=transforms,
                                            priors=priors)[0])
objective_with_grad = value_and_grad(tmp_func, argnum=0)



# By iterating minimize within a for cycle, we can save all intermediate results and set ending times
#start_time = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
num_fixed_points = Ns
num_trials = simParams['Ntrain']
batchnum = 0 
save_fname_params = "_%0.2d_fix_%0.3d_trials_batch_%0.2d" % (num_fixed_points, num_trials, batchnum)
save_fname = "Experiment_machens_results/machens_" + start_time + save_fname_params + ".pkl"
init_time = time.time()
max_time = 6.0*3600 # Maximum iteration time in seconds, break if reached
#all_results = []

for it in range(100):
    result = scipy.optimize.minimize(objective_with_grad, cur_pvec, jac=True, method='L-BFGS-B', bounds=bnds, callback=None,
                          options={'maxiter':10, 'disp':True})
    all_results.append(result)
    # Save the results
    with open(save_fname, 'wb') as f:
        pickle.dump([y, x,
                     all_results, 
                     init_paramvec, dict_ind, dict_shape, opt_params, 
                     bnds, transforms], f)
    cur_pvec = result.x
    cur_time = time.time()
    print([it, cur_time - init_time, result.fun])
    
    machens_callback_plot(cur_pvec)


    # Exit if maximum time is reached
    if ((cur_time - init_time) > max_time):
        print(["Maximum iteration time reached at iter", it])
        break

    if len(all_results)>=2:
        if (all_results[-1].fun - all_results[-2].fun) >= (-1e-2*num_trials):
            print(["Update did not improve objective function, stopping"])
            break

    



NameError: name 'y_train' is not defined

# Examine results

In [78]:
cur_pvec = result.x
cur_time = time.time()

In [80]:
machens_callback_plot(init_paramvec[opt_params])

In [79]:
machens_callback_plot(cur_pvec)

In [84]:

paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

# Unpack the usual parameters
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J, Sigma_s, Sigma_J)  = \
    paramdict.values()
    
np.set_printoptions(precision=8)
print np.concatenate([s.T, Sigma_s], axis=1)

[[ 0.29588147  0.32110281  0.08480658]
 [ 2.81525752  0.01415071  0.01147731]
 [ 1.3594525   1.45121535  0.00717799]
 [ 2.29829315  0.4561501   0.01637264]
 [-0.01348738  2.83927587  0.01487311]
 [ 0.23891016  2.47638252  0.01349222]]


In [89]:
paramdict

OrderedDict([('Sigma_eps', array([[ 0.02571329],
                     [ 0.0248558 ]])), ('mu_0_0', array([[ 0.30346966],
                     [ 0.3091754 ]])), ('Sigma_0_0', array([[ 0.05296571],
                     [ 0.03473274]])), ('C', array([[ 1.,  0.],
                     [ 0.,  1.]])), ('Sigma_nu', array([[  3.99820285e-06],
                     [  4.01740825e-05]])), ('z',
              array([[-0.58624354,  0.17730769,  0.94756844,  1.71539407,  2.45017884,
                       3.26963737, -0.58640159,  0.17930114,  0.95161552,  1.6929882 ,
                       2.48139546,  3.25890193, -0.58899315,  0.17605817,  0.91217041,
                       1.72867283,  2.47501511,  3.2482376 , -0.59139958,  0.15637211,
                       0.93725623,  1.69965124,  2.4793649 ,  3.24841084, -0.61248575,
                       0.17191987,  0.92642479,  1.71038425,  2.48090576,  3.24863282,
                      -0.61050156,  0.19577936,  0.94318176,  1.71318119,  2.48119203,
     

# Plotting individual trajectories and estimates

In [ ]:
%run "GPDM_direct_fixedpoints.ipynb"

In [ ]:
# Plot smoothed trajectories
obj, x_t1, x_t, sig_t1, sig_t, negll_all = time_full_iter(replace_params(all_results[-1].x, opt_params, init_paramvec), 
                                               y, dict_ind, dict_shape, log_transformed=log_transformed, ret_smoothed=True)


In [ ]:
np.unravel_index(np.argmin(negll_all), negll_all.shape)

In [ ]:
plt([plt_type.Histogram(x=np.squeeze(negll_all.flatten()))])

In [ ]:

def plot_trajectories(y,x,
                      x_t1, x_t, sig_t1, sig_t,
                      v,
                      ax_pixel = 700,
                      ax_range = (-0.5,3.5),
                      user_scale = 1.0,
                      marker_op=0.3):

    
    # Use proper scaling of the markers to represent standard deviation (or a fraction of it based on user-scale)
    std_scale = user_scale * (ax_pixel/(ax_range[1]-ax_range[0]))
    
    plots_by_run = [] 
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(y[0,:,v]), 
                      y=np.squeeze(y[1,:,v]), 
                      mode='lines',
                        name='y')
    )
    
#     plots_by_run.append(
#         plt_type.Scatter(x=np.squeeze(x[0,:,v]), 
#                       y=np.squeeze(x[1,:,v]), 
#                       mode='lines')
#     )

    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(x_t[0,:,v]), 
                      y=np.squeeze(x_t[1,:,v]), 
                      mode='lines',
                        name ='x_t')
    )
    
    plots_by_run.append(
        plt_type.Scatter(x=np.squeeze(x_t1[0,:,v]), 
                      y=np.squeeze(x_t1[1,:,v]), 
                      mode='markers',
                    marker=dict(size=np.squeeze(std_scale*np.sqrt(np.mean(sig_t1[[0,3],:,v], axis=0))),
                               opacity=marker_op),
                        name = 'x_t1')
    )
    
    
    layout1 = plt_type.Layout(
        width=ax_pixel, 
        height=ax_pixel,
        xaxis=dict(range=ax_range),
        yaxis=dict(range=ax_range)
    )
    
    plt(plt_type.Figure(data=plots_by_run, layout=layout1))


    layout2 = plt_type.Layout(
        height=ax_pixel,
        yaxis=dict(range=ax_range)
    )
    

    plt_data = [
        plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(y[0,:,v]), 
                      mode='lines', name='y_0'),
        
        plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(y[1,:,v]), 
                      mode='lines', name='y_1'),
    
                plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t[0,:,v]), 
                      mode='lines', name='xt_0'),
                plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t[1,:,v]), 
                      mode='lines', name='xt_1'),
            
            plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t1[0,:,v]), 
                      mode='lines+markers', name='xt1_0',
                            marker=dict(size=np.squeeze(std_scale*np.sqrt(sig_t1[0,:,v])),
                                       opacity=marker_op)),
        
                        plt_type.Scatter(x=np.squeeze(np.arange(y.shape[1])), 
                      y=np.squeeze(x_t1[1,:,v]), 
                      mode='lines+markers', name='xt1_1',
                            marker=dict(size=np.squeeze(std_scale*np.sqrt(sig_t1[-1,:,v])),
                                        opacity=marker_op)
                                        )
    ]
    
    plt(plt_type.Figure(data=plt_data, layout=layout2))

In [ ]:
plot_trajectories(y,x,
                      x_t1, x_t, sig_t1, sig_t,
                      3)

In [ ]:
tcur = 5; v = 17

In [ ]:
Sigma_eps

In [ ]:
sig_t1[:,tcur:(tcur+1),v]

In [ ]:
sig_t1[:,tcur:(tcur+1),v]

In [ ]:
Sigma_t_t1 = np.reshape(sig_t1[:,tcur:(tcur+1),v],(2,2))

In [ ]:
mu_t_t1 = x_t1[:,tcur:(tcur+1),v]

In [ ]:
y_t = y[:,tcur:(tcur+1),v]

In [ ]:
mu_gauss = np.dot(C, mu_t_t1) 
var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())

# Check for condition number, if high compared to expected precision in y (Sigma_nu), use pseudo-inverse and pseudo-determinant
prec = np.mean(np.log(1./Sigma_nu))
prec = prec-1
if np.log(np.linalg.cond(var_gauss)) > prec:
    inv_var_gauss = np.linalg.pinv(var_gauss, rcond=(1./prec))
    ldet = np.abs(np.linalg.eigvals(var_gauss))
    ldet = np.sum(np.log(ldet[np.max(ldet)>(1./prec)]))
else:    
    inv_var_gauss = np.linalg.inv(var_gauss)
    ldet = np.linalg.slogdet(var_gauss)[1]

D = mu_t_t1.shape[1]*1.0

log_marg_ll = (
    - 1.0* D/2.0*np.log(2*np.pi)
    - 1.0/2.0*ldet
    - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
)

In [ ]:
np.abs(np.linalg.eigvals(var_gauss))/np.max(np.abs(np.linalg.eigvals(var_gauss)))

In [ ]:
prec = np.mean(np.log(1./Sigma_nu))

In [ ]:
np.log(np.linalg.cond(var_gauss))

In [ ]:
prec

In [ ]:
np.log(np.linalg.cond(var_gauss)) > prec

In [ ]:
log_marg_ll

In [ ]:
inv_var_gauss = np.linalg.inv(var_gauss)
ldet = np.linalg.slogdet(var_gauss)[1]

In [ ]:
np.linalg.slogdet(var_gauss)

In [ ]:
- 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss))

In [ ]:
ldet = np.linalg.svd(var_gauss, compute_uv=False)
ldet = np.sum(np.log(ldet[ldet>(1./prec)]))

In [ ]:
np.linalg.svd(var_gauss, compute_uv=False)

In [ ]:
ldet

In [ ]:
ldet = np.linalg.svd(var_gauss, compute_uv=False)
ldet = np.sum(np.log(ldet[ldet>(1./prec)]))
- 1.0/2.0*ldet

In [ ]:
inv_var_gaussa

In [ ]:
y_t - mu_gauss

In [ ]:
np.linalg.svd(var_gauss, compute_uv=False)

In [ ]:
ldet[ldet>(1./prec)]

In [ ]:
log_marg_ll

In [ ]:
nlog_marg_ll(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y_t)

In [ ]:
np.diag(Sigma_nu.flatten())

In [ ]:
np.dot(np.dot(C, Sigma_t_t1), C.T)

In [ ]:
np.linalg.cond(Sigma_t_t1 + np.diag(Sigma_nu.flatten()))

In [ ]:
replace_params(all_results[-1].x, opt_params, init_paramvec)[dict_ind['Sigma_eps']]

In [ ]:
prec = np.mean(np.log(1./Sigma_nu))

In [ ]:
prec = np.mean(np.log(1./Sigma_nu))
np.log(np.linalg.cond(var_gauss))

In [ ]:
inv_var_gauss = np.linalg.inv(var_gauss)
ldet = np.linalg.slogdet(var_gauss)[1]
print(inv_var_gauss)
print(ldet)

In [ ]:
np.linalg.svd(var_gauss, compute_uv=False)**2

In [ ]:
np.linalg.eigvals(var_gauss)

In [ ]:
np.linalg.cond(var_gauss)

In [ ]:
1./prec

In [ ]:
inv_var_gauss = np.linalg.pinv(var_gauss, rcond=(1./prec))
ldet = np.abs(np.linalg.eigvals(var_gauss))
ldet = np.sum(np.log(ldet[(ldet/np.max(ldet))>(1./prec)]))
print(inv_var_gauss)
print(ldet)

In [ ]:
D = 2.0

In [ ]:
log_marg_ll = (
    - 1.0* D/2.0*np.log(2*np.pi)
    - 1.0/2.0*np.linalg.slogdet(var_gauss)[1] 
    - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
)
print log_marg_ll

In [ ]:
mu_gauss = np.dot(C, mu_t_t1) 
var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())

inv_var_gauss = np.linalg.inv(var_gauss)

D = mu_t_t1.shape[1]*1.0

log_marg_ll = (
    - 1.0* D/2.0*np.log(2*np.pi)
    - 1.0/2.0*np.linalg.slogdet(var_gauss)[1] 
    - 1.0/2.0*np.dot(np.dot( (y_t.T - mu_gauss.T), inv_var_gauss), (y_t - mu_gauss) )  
)

In [ ]:
var_gauss = np.dot(np.dot(C, Sigma_t_t1), C.T) + np.diag(Sigma_nu.flatten())

In [ ]:
np.linalg.cond(var_gauss)

In [ ]:
np.exp(np.linalg.slogdet(var_gauss)[1])

In [ ]:
Sigma_t_t1

In [ ]:
(y_t.T - mu_gauss.T)

In [ ]:
np.linalg.slogdet(var_gauss)

In [ ]:
proj_var = np.dot(C, Sigma_t_t1)
proj_inv = np.linalg.inv(np.dot(proj_var, C.T) + np.diag(Sigma_nu.flatten()))
proj_back = np.dot(Sigma_t_t1, np.dot(C.T, proj_inv))

mu_t_t = mu_t_t1 + np.dot(proj_back, (y_t - np.dot(C, mu_t_t1)))

Sigma_t_t = Sigma_t_t1 - np.dot(proj_back, proj_var)

In [ ]:
y_t - np.dot(C, mu_t_t1)

In [ ]:
proj_back

In [ ]:
np.diag(Sigma_nu.flatten())

In [ ]:
np.linalg.cond(np.dot(proj_var, C.T)+ np.diag(Sigma_nu.flatten()))

In [ ]:
proj_inv

In [ ]:
y_t

In [ ]:
mu_t_t1

In [ ]:
Sigma_nu

In [ ]:
machens_callback_plot(cur_pvec)

In [ ]:
with open('machens_20171206T072804.pkl', 'r') as f:
    [y, all_results, init_paramvec, dict_ind, dict_shape, opt_params, bnds, log_transformed] = pickle.load(f)

In [ ]:
cur_pvec = all_results[-1].x

In [ ]:
machens_callback_plot(init_paramvec[opt_params])

In [ ]:
for i in range(len(all_results)):
    machens_callback_plot(all_results[i].x)

In [ ]:
cur_pvec

In [ ]:
len(all_results)

# Evaluation metrics

In [ ]:
with open('machens_20171207T172326.pkl', 'r') as f:
    [y, x, simParams, 
     all_results, init_paramvec, 
     dict_ind, dict_shape, opt_params, 
     bnds, log_transformed] = pickle.load(f)

In [ ]:
# Good runs

# # Ns = 1
# 'machens_20171207T172326.pkl'

# # Ns = 2, Nz = 16

# 'machens_20171206T072804.pkl'

# # Ns = 5, Nz = 36

# 'machens_20171207T041533.pkl' - Learned J
# 'machens_20171207T054327.pkl' - Fixed J

In [ ]:
# Load held out data
np.random.seed(simParams['seed'])
x_test = tmp['y_python'][:,0::simParams['delta_t'],simParams['Ntrain']:]*simParams['rescale'] # Rescaling to reduce numerical instability
y_test = x_test + np.sqrt(simParams['Sigma_y'])*np.random.randn(*x_test.shape)
y_test.shape

### One step ahead given data

In [ ]:
# Perform linear AR(1) on y as baseline
Dy = y_test.shape[0]
y_t = y[:,1:,:]
y_t1 = y[:,:-1,:]

y_t_rsh = np.reshape(y_t, (y.shape[0], -1)).T
y_t1_rsh = np.reshape(y_t1, (y.shape[0], -1)).T

# Get least squares linear regression weights from original dataset
weights_AR = np.dot(np.dot(np.linalg.inv(np.dot(y_t1_rsh.T, y_t1_rsh)), y_t1_rsh.T), y_t_rsh)

# Get optimal linear estimate of y_tt given the y_t-1 vector
y_test_t1_rsh = np.reshape(y_test[:,:-1,:], (y.shape[0], -1)).T
y_test_t_hat = np.dot(y_test_t1_rsh, weights_AR)

y_test_t_hat = np.reshape(y_test_t_hat.T, (y_test.shape[0], y_test.shape[1]-1, y_test.shape[2]))
y_test_t_hat = np.concatenate([np.zeros((y_test.shape[0],1,y_test.shape[2])), y_test_t_hat], axis=1)
y_test_t_hat.shape

In [ ]:
Sigma_eps

In [ ]:
weights_AR

In [ ]:
# Run the smoothing on this data, then check for RMSE against true x
# res = [obj, x_t1, x_t, sig_t1, sig_t, negll_all]
res_heldout_init = time_full_iter(init_paramvec, 
                                               y_test, dict_ind, dict_shape, log_transformed=log_transformed, ret_smoothed=True)
res_heldout_final = time_full_iter(replace_params(all_results[-1].x, opt_params, init_paramvec), 
                                               y_test, dict_ind, dict_shape, log_transformed=log_transformed, ret_smoothed=True)


In [ ]:
# Get error values between x_t1 and true x
RMSE_AR = np.sqrt(np.mean(np.mean((y_test_t_hat-x_test)**2, axis=2), axis=0))
RMSE_final = np.sqrt(np.mean(np.mean((res_heldout_final[1]-x_test)**2, axis=2), axis=0))
RMSE_init = np.sqrt(np.mean(np.mean((res_heldout_init[1]-x_test)**2, axis=2), axis=0))

plt([
        plt_type.Scatter(y=np.squeeze(RMSE_final), name='RMSE_final'),
        plt_type.Scatter(y=np.squeeze(RMSE_init), name='RMSE_init'),
        plt_type.Scatter(y=np.squeeze(RMSE_AR), name='RMSE_AR')
    ])

### Run forward given initial data point on each trial

In [ ]:
# Get estimate of true variance (around fixed point)
x_var_true = np.var(np.reshape(x[:,x.shape[1]-10:x.shape[1],:] - x[:,x.shape[1]-11:x.shape[1]-1,:], (x.shape[0], -1)).T, axis = 0)
x_var_true = x_var_true[:,None]
x_var_true

In [ ]:
# Copied from time_full_iter

Dy = y.shape[0]
T = y.shape[1]
Ny = y.shape[2]
    
paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict_ind = dict_ind
paramdict_shape = dict_shape

# Transform the parameters back from log-space
if log_transformed is not None:
    inds = (np.arange(paramvec.shape[0]))
    inds = np.setdiff1d(inds,log_transformed)
    out = np.concatenate([np.exp(paramvec[log_transformed]), paramvec[inds]])
    out = out[np.argsort(np.concatenate([log_transformed, inds]))]
    paramvec = out


# Unpack the usual parameters
param_tuple = vec_to_params(paramvec, paramdict_ind, paramdict_shape)
(Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
    tuple(list(param_tuple)[:12])

if np.any(np.isnan(lengthscales)):
    set_trace()

# Deal with the extra possible parameters
Sigma_s = None; Sigma_J=None;
if 'Sigma_s' in paramdict_ind.keys():
    Sigma_s = np.reshape(paramvec[paramdict_ind['Sigma_s']], paramdict_shape['Sigma_s'])
if 'Sigma_J' in paramdict_ind.keys():
    Sigma_J = np.reshape(paramvec[paramdict_ind['Sigma_J']], paramdict_shape['Sigma_J'])

L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u = Sigma_u, sig_s=Sigma_s, sig_J=Sigma_J)

# Collect smoothed latent trajectories
x_all_t1 = np.zeros(x_test.shape)
sig_all_t1 = np.zeros((x_test.shape[0]**2, x_test.shape[1], x_test.shape[2]))

for n in range(x_test.shape[2]):
    mu_t1_t1 = mu_0_0
    Sigma_t1_t1 = Sigma_0_0
        
    for t in range(x_test.shape[1]):
        mu_t_t1, Sigma_t_t1 = update_t_t1(mu_t1_t1, Sigma_t1_t1, L, targets, kernel_variance, 
                                          Sigma_eps, z, u, lengthscales, s, J)
        
        x_all_t1[:,t,n] = mu_t_t1.flatten()            
        sig_all_t1[:,t,n] = np.reshape(Sigma_t_t1,(1,-1))
        
        cur_nll_term, mu_t1_t1, Sigma_t1_t1 = update_t_t(mu_t_t1, Sigma_t_t1, C, Sigma_nu, y_test[:,t:(t+1),n])
        #print [mu_t1_t1, y_test[:,t:(t+1),n]]
        Sigma_t1_t1 = np.diag(Sigma_t1_t1)[:,None]
        if t>2:
            # Integrate the first Tau data points, but then no more data available (ignore mu_t1_t1, do we add noise?)
            mu_t1_t1 = mu_t_t1 # + np.sqrt(Sigma_eps)*np.random.randn(*mu_t_t1.shape)

        # Check "condition number" and add diag term to correct if needed            
        if (np.min(Sigma_t1_t1)<1e-6):
            Sigma_t1_t1 = Sigma_nu

In [ ]:
plot_trajectories(y_test,x_test,
                      x_all_t1, x_all_t1, sig_all_t1, sig_all_t1,
                      1, marker_op=0.3)

In [ ]:
machens_callback_plot(all_results[-1].x)